# Problème 2 : Cryptographie

Un **chiffrement** est un procédé qui transforme un message en un autre message (le message chiffré), de manière à rendre impossible, en principe, la reconstitution du message original à partir du message chiffré, sauf à connaître la **clé** de déchiffrement, c'est-à-dire le ou les paramètres nécessaires à l'opération de déchiffrement. Notons au passage que *chiffrer* ne signifie donc pas *transformer en chiffres*.

Dans ce problème, nous allons étudier la méthode de chiffrement de Hill. L'objectif est de programmer les opérations de chiffrement et de déchiffrement, ainsi qu'une attaque par force brute.

La méthode de Hill fait appel à des concepts d'arithmétique et en particulier à la notion d'**inverse modulaire**.

## A. Rappels d'arithmétique

Les notions de **division euclidienne**, de **multiple**, de **diviseur** et de **PGCD** sont supposées connues (voir le cours de L1 ou le document d'accompagnement sur l'arithmétique si nécessaire).

* On appelle **reste de $a$ modulo $b$** le reste de la division euclidienne de $a$ par $b$. On le note $a \operatorname{mod} b$. Quel opérateur Python permet de calculer ce reste ? A l'aide de cet opérateur, vérifier que $60 \bmod 26 = 8$, puis calculer $111 \operatorname{mod} 26$.

In [10]:
print(60 % 26)
print(111 % 26)

8
7


* On appelle **quotient entier de $a$ par $b$** le quotient de la division euclidienne de $a$ par $b$. Quel opérateur Python permet de calculer ce quotient ? A l'aide de cet opérateur, vérifier que le quotient entier de $60$ par $26$ est $2$, puis calculer le quotient entier de $111$ par $9$.

In [11]:
print(60//26)
print(111//9)

2
12


* Quelle fonction Python (du module `math`) permet de calculer le PGCD de deux entiers ? A l'aide de cette fonction, calculer le PGCD de 196 et 252, puis le PGCD de 196 et 225.

In [12]:
from math import gcd
print(gcd(196, 252))
print(gcd(196, 225))

28
1


On dit que deux entiers sont **premiers entre eux** si leur PGCD est égal à 1. (Attention : cette notion est distincte de la notion de nombre premier.)

* A l'aide d'un (court) programme Python, déterminer tous les nombres compris entre $1$ et $30$ qui sont premiers avec $32$ (vous devez obtenir $1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29$ ; même chose avec tous les nombres compris entre $1$ et $30$ qui sont premiers avec $10$.

In [13]:
print("Premier avec 32 : ")
for i in range(30):
    if gcd(i, 32) == 1 :
        print(i, end=", ")
        
print("\nPremier avec 10 :")        
for i in range(30):
    if gcd(i, 10) == 1 :
        print(i, end=", ")


Premier avec 32 : 
1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 
Premier avec 10 :
1, 3, 7, 9, 11, 13, 17, 19, 21, 23, 27, 29, 

**Définition.** Soit $n\in \mathbb{N}^*$. On dit que deux entiers $a$ et $b$ sont **congrus modulo $n$** si $n$ divise $b-a$. On note cette relation $a \equiv b \pmod n$. 

Par exemple, $4 \equiv 10 \pmod 3$.

En particulier $a \equiv 0 \pmod n$ signifie que $n$ divise $a$.

Les caractérisations suivantes de la relation de congruence sont souvent utiles.

**Proposition.** Deux entiers $a$ et $b$ sont congrus modulo $n$ si et seulement s'il existe un entier $k$ tel que $a = b + kn$.

**Proposition.** Deux entiers $a$ et $b$ sont congrus modulo $n$ si et seulement s'ils ont le même reste modulo $n$.

* Parmi les relations suivantes, lequelles sont vraies ?

    * $ 23 \equiv 9 \pmod 7$

    * $ 41 \equiv 21 \pmod 8$

    * $ -5 \equiv 7 \pmod 6$

    * $ -16 \equiv 16 \pmod{11}$

<span style = "color : blue;"> 
Les relations vraies sont : $ -5 \equiv 7 \pmod 6 $ et $ 23 \equiv 9 \pmod{7} $ 
</span>

**Proposition.** La relation de congruence modulo $n$ est une relation d'équivalence. 

**Proposition.** Si  $a \equiv b \pmod n$ et  $c \equiv d \pmod n$, alors  $a +c \equiv b+d \pmod n$ et $ac \equiv bd \pmod n$.

## B. Inverse modulaire d'un entier

**Définition.** Soit $n \in \mathbb{N}^*$. Soit $a \in \mathbb{Z}$. On dit que $b \in \mathbb{Z}$ est un **inverse de $a$ pour la multiplication modulo $n$** (ou plus simplement un **inverse de $a$ modulo $n$**) si :

$$
a b \equiv 1 \pmod n.
$$

Si un nombre admet un inverse modulo $n$, on dit qu'il est **inversible** modulo $n$.

**Exemple.** Le nombre $8$ est inversible modulo $11$. Le nombre $7$ est un inverse de $8$ car $7 \times 8 \equiv 56 \equiv 1 \pmod{11}$. Les entiers $18$ et $-4$ sont aussi des inverses de $8$ modulo $7$.

Attention, cet inverse n'existe pas toujours. Par exemple $2$ n'a pas d'inverse modulo $6$.

Voici quelques résultats fondamentaux sur les inverse modulaires.

**Proposition.** Un entier $a$ est inversible modulo $n$ si et seulement si $a$ et $n$ sont premiers entre eux.


**Proposition.** Si $x$ est un inverse de $a$ modulo $n$ alors tous les nombres $y$ tels que $x \equiv y \pmod n$ sont des inverses de $a$ modulo $n$. En particulier $x \operatorname{mod} n$ est un inverse de $a$ modulo $n$. C'est l'unique inverse compris entre $1$ et $n-1$.

### B.1 Un algorithme naïf

Pour trouver un inverse de $a$ modulo $n$, on peut procéder ainsi.

*On parcourt les nombres de $1$ à $n-1$ et on teste chaque nombre (en calculant son produit avec $a$ modulo $n$) pour voir s'il s'agit d'un inverse modulo $n$. Dès qu'un nombre convient, on le renvoie et on s'arrête. Si aucun nombre ne convient, on renvoie un message d'erreur.*

* Ecrire une fonction `invmod` qui prend en paramètre un entier $a$ et un entier $n$, et qui met en œuvre l'algorithme de calcul d'inverse décrit ci-dessus. Pour renvoyer une erreur, utiliser l'instruction `raise ValueError('Nombre non inversible')`. Tester cette fonction sur les exemples suivants.

```py
>>> invmod(8, 11)
7
>>> invmod(2, 6)
Traceback (most recent call last)
 ...
ValueError: Nombre non inversible
```

In [14]:
def invmod(a, n) :
    for i in range(n-1) :
        if (i*a)%n == 1 :
            return i
    raise ValueError("Nombre non inversible")
    
invmod(8, 11)
#invmod(2, 6) ligne qui cherche l'erreur

7

### B.2 Algorithme d'Euclide étendu

L'algorithme précédent n'est pas très efficace quand $n$ devient grand. Il existe une meilleure façon de déterminer un inverse modulaire basée sur l'**algorithme d'Euclide étendu**.

L'**algorithme d'Euclide** est l'algorithme de référence pour déterminer le PGCD de deux nombres.

Il repose sur le calcul des termes des suites récurrentes $(a_n)_{n\in\mathbb{N}}$ et $(b_n)_{n\in\mathbb{N}}$ définies par

$$ \left\{
\begin{align*}
& a_0 = a \\
& b_0 = b\\
& a_{n+1} = b_n \\
& b_{n+1} = a_n \operatorname{mod} b_n \\
\end{align*}
\right.
$$

Dès que que $b_n$ vaut $0$, l'algorithme s'arrête et renvoie $a_n$, qui est alors égal au PGCD de $a$ et $b$.

La fonction suivante propose une implémentation de l'algorithme d'Euclide.

```py
def euclide(a, b):
    while b != 0:
        a, b = b , a % b
    return a
```

L'algorithme d'Euclide étendu est une variante de l'algorithme d'Euclide qui, en plus du PGCD de $a$ et $b$, renvoie deux coefficients $u$ et $v$ tels que :

$$
au+bv = \text{PGCD}(a, b)
$$

**Remarque.** L'existence de tels coefficients $u$ et $v$ n'a rien d'évident à première vue. Elle est en fait garantie par un théorème (le **théorème de Bezout**).

En plus du calcul des suites $(a_n)_{n\in\mathbb{N}}$ et $(b_n)_{n\in\mathbb{N}}$, l'algorithme d'Euclide étendu nécessite le calcul des suites $(u_n)_{n\in\mathbb{N}}$ et $(v_n)_{n\in\mathbb{N}}$ définies par

$$ \left\{
\begin{align*}
& u_0 = 1,\ u_1 = 0 \\
& v_0 = 0,\ v_1 = 1 \\
& u_{n+1} = u_{n-1} - q_n u_n \\
& v_{n+1} = v_{n-1} - q_n v_n \\
\end{align*}
\right.
$$

où $q_n$ désigne le quotient entier de $a_n$ par $b_n$.

Les suites $(u_n)_{n\in\mathbb{N}}$ et $(v_n)_{n\in\mathbb{N}}$ sont des suites récurrentes d'ordre 2. Comme l'algorithme d'Euclide, l'algorithme d'Euclide étendu s'arrête dès que que $b_n$ vaut $0$. Il renvoie $a_n$, qui est égal à $\text{PGCD}(a, b)$, ainsi que $u_n$ et $v_n$, qui vérifient $au_n+bv_n = \text{PGCD}(a, b)$. 

* Ecrire une fonction `euclide_etendu` qui met en œuvre l'algorithme d'Euclide étendu. (Voir le document *Suites récurrentes* pour la programmation du calcul des termes d'une suite récurrente d'ordre 2.)

In [35]:
def euclide_etendu(a, b):
    
    u_0, u_1 = 1, 0
    v_0, v_1 = 0, 1  
    
    while b!= 0 :
        
        q = a//b
        
        u_1, u_0 = u_0 - q*u_1, u_1
        v_1, v_0 = v_0 - q*v_1, v_1
        
        a, b = b, a % b
        
    return a, u_0, v_0

L'algorithme d'Euclide étendu permet de calculer l'inverse de $a$ modulo $n$. En appliquant cet algorithme aux nombres $a$ et $n$, on obtient en effet deux coefficients $u$ et $v$ tels que :

$$
a u + nv = 1.
$$

Par définition de la relation de congruence, cela signifie que

$$
a u \equiv 1 \pmod n.
$$

Le nombre $u$ est donc un inverse de $a$ modulo $n$. 

Le nombre $u$ n'est pas nécessairement l'inverse compris entre $1$  et $n-1$. Pour obtenir cet inverse, il suffit de prendre le nombre $u \operatorname{mod} n$.

* Tester la fonction `euclide_etendu` sur un exemple.

In [36]:
euclide_etendu(8, 11)

(1, -4, 3)

* Ecrire une fonction `invmod2` qui calcule l'inverse modulaire en faisant appel à la fonction `euclide_etendu`. Comme `invmod`, cette fonction renverra une erreur si l'inverse n'existe pas.

In [17]:
def invmod2(a, b, n):
    
    a, u, v = euclide_etendu(a, b)
    if u % n == 1 :
        return u
    raise ValueError("Nombre non inversible")

* Tester la fonction `invmod2` sur un exemple.

In [40]:
#invmod2(8, 1, 11)

ValueError: Nombre non inversible

## C. Inverse modulaire d'une matrice

**Définition.** On dit que deux matrices (ou deux vecteurs) sont congrus modulo $n$ si tous leurs coefficients sont congrus modulo $n$ deux à deux. On utilise la même notation que pour les entiers.

Par exemple,

$$
\begin{pmatrix}
7 & 3 \\ 10 & 1
\end{pmatrix}
\equiv
\begin{pmatrix}
2 & 8 \\ 0 & 16
\end{pmatrix}
\pmod 5
$$

Soit $A$ une matrice (ou un vecteur). On note $A \operatorname{mod} n$ la matrice (ou le vecteur) obtenue quand on prend le reste modulo $n$ des coefficients de $A$.

Par exemple,
$$
\begin{pmatrix}
7 & 3 \\ 10 & 1
\end{pmatrix}
\operatorname{mod} 5
=
\begin{pmatrix}
2 & 3 \\ 0 & 1
\end{pmatrix}
$$

**Définition.** Soit $n \in \mathbb{N}^*$. Soit $A$ une matrice $2\times 2$ à coefficients entiers. On dit qu'une matrice $2\times 2$ à coefficients entiers $B$ est une **matrice inverse de $A$ pour la multiplication modulo $n$** (ou plus simplement un **matrice inverse de $A$ modulo $n$**) si :

$$
A B \equiv \begin{pmatrix}
1 & 0 \\ 0 & 1
\end{pmatrix} \pmod n.
$$

Si une matrice admet une matrice inverse modulo $n$, on dit qu'elle est **inversible** modulo $n$.

**Proposition.** Si $M$ est une matrice inverse de $A$ modulo $n$ alors toutes les matrices $N$ telles que $M \equiv N \pmod n$ sont des matrices inverses de $A$ modulo $n$. En particulier $M \operatorname{mod} n$ est une matrice inverse de $A$ modulo $n$. C'est l'unique matrice inverse dont tous les coefficients sont compris entre $0$ et $n-1$.

On définit le **déterminant** d'une matrice $\begin{pmatrix}
a & b \\ c & d
\end{pmatrix}$ comme la quantité $ad-bc$.

* Soit $M = \begin{pmatrix}
a & b \\ c & d
\end{pmatrix}$. On note $\delta$ son déterminant. On suppose que $\delta$ est inversible modulo $n$ et on note $\gamma$ un inverse de $\delta$. Vérifier que la matrice

$$
B = \gamma \begin{pmatrix}
d & -b \\ -c & a
\end{pmatrix}
$$

est une matrice inverse de $M$ modulo $n$.

-- *Écrire la réponse ici.* --

Nous avons ainsi montré qu'une matrice est inversible modulo $n$ si son déterminant est inversible modulo $n$. Nous admettons que la réciproque est vraie (autrement dit que c'est un "si et seulement si").

*  Ecrire une fonction `det` qui calcule le déterminant d'une matrice $2 \times 2$.

In [82]:
def det(matrice) :
    a, b, c , d = matrice[0][0], matrice[0][1], matrice[1][0], matrice[1][1] 
    return (a * d) - (b * c)

d = det([[2, 1], [1, 3]])

* Ecrire une fonction `invmatmod` qui prend en paramètres une matrice `M` (de taille $2 \times 2$) et un entier `n`, et qui renvoie une matrice inverse modulo $n$ dont tous les coefficients sont compris entre $0$ et $n-1$. 

In [86]:
def invmatmod(matrice, n):
    deter = det(matrice)
    inv_deter = deter % n
    
    a, b, c , d = matrice[0][0], matrice[0][1], matrice[1][0], matrice[1][1] 
    t_M = [[d, -b], [-c, a]]
    
    w, x, y, z = t_M[0][0], t_M[0][1], t_M[1][0], t_M[1][1]
    
    return [[inv_deter * w, inv_deter * x], [inv_deter * y, inv_deter * z]]
    

[[15, -5], [-5, 10]]

* Tester la fonction `invmatmod` sur les exemples suivants.
```py
>>> invmatmod([[2, 1], [1, 3]], 26)
[[11, 5], [5, 16]]
>>> invmatmod([[10, 9], [21, 5]], 26)
[[11, 1], [11, 22]]
```

In [87]:
invmatmod([[2, 1], [1, 3]], 26)

[[85, -153], [-357, 170]]

* Grâce à `invmatmod`, déterminer une matrice inverse modulo $26$ de $\begin{pmatrix} 2 & 1 \\ 5 & 12 \end{pmatrix}$.

## D. Chiffrement de Hill

Par souci de simplicité, dans cette section, nous considérons que les messages à chiffrer sont écrits uniquement avec les 26 lettres de l'alphabet latin, en majuscule, sans [signes diacritiques](https://fr.wikipedia.org/wiki/Diacritiques_utilis%C3%A9s_en_fran%C3%A7ais), sans espace et sans ponctuation.


La **méthode de Hill** consiste à découper le message en blocs de deux lettres et à remplacer chaque bloc de deux lettres par un autre bloc de deux lettres, selon une procédure impliquant un produit matrice-vecteur. La matrice utilisée (une matrice $2\times 2$ à coefficients dans $\{0,\ldots,25\}$) est appelée matrice de chiffrement.

Voyons sur un exemple comment fonctionne cette méthode. 

La matrice de chiffrement choisie est la matrice

$$
M = \begin{pmatrix}2 & 19 \\ 5 & 12\end{pmatrix}.
$$

Le message à chiffrer est $\texttt{MATHEMATIQUE}$.

**Etape 1.** Le texte doit d'abord être divisé en blocs de deux lettres.

$$\texttt{MA TH EM AT IQ UE}$$

Puis chaque bloc de deux lettres est converti en vecteur de deux nombres (A devenant 0, B devenant 1, C devenant 2, etc.)

$$(12, 0)\ (19, 7)\ (4, 12)\ (0, 19)\ (8, 16)\ (20, 4)$$

**Etape 2.** On définit l'application 

$$
g : (x, y) \mapsto M \begin{pmatrix}
x \\ y
\end{pmatrix} \operatorname{mod} 26,
$$

On transforme chaque vecteur en lui appliquant la transformation  $g$. 

Pour le vecteur $(12, 0)$, la transformation donne :

$$
\varphi (12, 0) = \begin{pmatrix}2 & 19 \\ 5 & 12\end{pmatrix} \begin{pmatrix}12 \\ 0\end{pmatrix} \operatorname{mod} 26 = \begin{pmatrix}24 \\ 60\end{pmatrix} \operatorname{mod} 26 = \begin{pmatrix}24 \\ 8\end{pmatrix}.
$$

En appliquant la même transformation à tous les vecteurs, on trouve :

$$
(24, 8)\ (15, 23)\ (2, 8)\ (23, 20)\ (8, 24)\ (12, 18)\ 
$$


**Etape 3.** Il reste à convertir les vecteurs de nombres en blocs de lettres (0 devenant A, 1 devenant B, 2 devenant C, etc.) Finalement, le texte chiffré est :

$$
\texttt{YIPXCIXUIYMS}
$$

Nous choisissons de représenter informatiquement les vecteurs par des listes et les matrices par des listes de listes (chaque sous-liste contenant les coefficients d'une ligne).

Par exemple, le vecteur $(2, 1)$ est représenté par la liste `[2, 1]` et la matrice $\begin{pmatrix}1 & 0 \\ 2 & -3\end{pmatrix}$ est représentée par `[[1, 0], [2, -3]]`.

* Ecrire une fonction `mulmat` qui renvoie le produit de deux matrices $2\times 2$.

```py
>>> A = [[2, 1], [1, 3]]
>>> B = [[1, 2], [3 ,0]]
>>> mulmat(A, B)
[[5, 4], [10, 2]]
```

In [22]:
def mulmat(mat1, mat2) :
    
    a, b, c, d = mat1[0][0], mat1[0][1], mat1[1][0], mat1[1][1]
    w, x, y, z = mat2[0][0], mat2[0][1], mat2[1][0], mat2[1][1]
    
    m1 = a * w + b * y 
    m2 = a * x + b * z
    m3 = c * w + d * y
    m4 = c * x + c * z
    
    return [[m1, m2], [m3, m4]]

mulmat([[2, 1], [1, 3]], [[1, 2], [3 ,0]])

[[5, 4], [10, 2]]

* Ecrire une fonction `mulmatvec` qui calcule le produit d'une matrice $2\times 2$ et d'un vecteur de taille $2$.


```py
>>> A = [[2, 1], [1, 3]]
>>> v = [1, 2]
>>> mulmat(A, v)
[4, 7]
```

In [27]:
def multmatvec(mat, vect) :
    
    a, b, c, d = mat[0][0], mat[0][1], mat[1][0], mat[1][1]
    x, y = vect[0], vect[1]
    
    return [a * x + b * y, c * x + d * y]

multmatvec([[2, 1], [1, 3]], [1, 2])

[4, 7]

* Ecrire une fonction `cvn`qui transforme un caractère en nombre (`"A"` est transformé en `0`, `"B"` en `1`, etc.) et une fonctions `nvc`qui transforme un nombre en caractère (`0` est transformé en `"A"`, `1` en `"B"`, etc.) Indication : on pourra utiliser les fonctions prédéfinies `chr` et `ord` (voir documentation Python).

```py
>>> cvn("P")
15
>>> nvc(16)
'Q'
```

In [69]:
def cvn(caract) :
    return ord(caract) - 65

def nvc(nombre):
    return chr(nombre + 65)

print(cvn('P'))
print(cvn('a'))
print(nvc(16))
print(nvc(2))

15
32
Q
C


- Ecrire une fonction `chiffre` qui chiffre un texte par la méthode de Hill. Elle prendra en paramètres `M`, la matrice de chiffrement, et `text`, le texte à chiffrer. Si le texte à chiffrer possède un nombre impair de caractères, on ajoutera un caractère final tiré aléatoirement (en utilisant la fonction `randint` du module `random` par exemple). 

In [100]:
from random import randint

def chiffre(text):
    
    taille = len(text)
    if taille % 2 != 0 :
        text += nvc(randint(0, 25)) # ajoute un caractère aléatoire final
    lst = []
    for i in range(0, taille, 2):
        lst.append((text[i], text[i+1]))
        
    for elem in lst :
            print(elem)
        
    return lst

- Tester la fonction `chiffre` sur la chaîne `"MATHEMATIQUE"`.

In [101]:
chiffre("MATHEMATIQUE")

('M', 'A')
('T', 'H')
('E', 'M')
('A', 'T')
('I', 'Q')
('U', 'E')


[('M', 'A'), ('T', 'H'), ('E', 'M'), ('A', 'T'), ('I', 'Q'), ('U', 'E')]

## E. Matrices de chiffrement admissibles et déchiffrement

Un texte chiffré par une méthode de chiffrement doit, cela va sans dire, pouvoir être déchiffré. Pour cela, chaque texte chiffré doit correspondre à un et un seul texte initial. Autrement dit, la procédure de chiffrement doit être bijective. Pour la méthode de Hill, cela signifie que la fonction $g$ (définie plus haut) doit être bijective sur l'ensemble des couples de nombres compris entre $0$ et $25$.

Tous les choix de matrice $M$ ne conduisent pas à des fonctions $g$ bijectives. Par exemple, en prenant

$$
M = \begin{pmatrix}2 & 18 \\ 6 & 12\end{pmatrix},
$$

on trouve que 
$$
g(21, 4) = (10, 18) = g(8, 17).
$$

L'objectif de cette partie est de déterminer les matrices de chiffrement admissibles (celles qui correspondent à des fonctions $g$ bijectives).

* Soit $M$ une matrice inversible modulo 26. Soit $W$ une matrice inverse modulo $26$ de $M$. On définit l'application

$$
h : \begin{pmatrix}
x \\ y
\end{pmatrix} \mapsto W \begin{pmatrix}
x \\ y
\end{pmatrix} \operatorname{mod} 26
$$

Vérifier que, pour tous $x$, $y$ compris entre $0$ et $25$, $g \circ h (x, y) = (x, y)$, ce qui prouve que $g$ est bijective et que $h$ est l'application réciproque de $g$.

-- *Écrire la réponse ici.* --

Nous avons ainsi montré que l'application $g$ est bijective si la matrice de chiffrement est inversible modulo $26$. Nous admettons que la réciproque est vraie. Nous avons aussi trouvé une expression explicite pour la fonction réciproque de $g$.

L'opération de déchiffrement est similaire à l'opération de chiffrement.

* **Etape 1.** Conversion des blocs de lettres en vecteurs de nombres.

* **Etape 2.** Application de la fonction réciproque de $g$ sur les vecteurs.

* **Etape 3.** Conversion des vecteurs de nombres en blocs de lettres.


- Ecrire une fonction `dechiffre` qui déchiffre un text chiffré par chiffrement de Hill. Elle prendra en paramètres `M`, la matrice de chiffrement, et `text`, le texte à déchiffrer.

- En utilisant la fonction `dechiffre`, déchiffrer le texte `TXOFANJMYAXLTXLPBVUN` sachant qu'il a été chiffré avec la matrice $\begin{pmatrix} 1 & 2 \\ 1 & 3 \end{pmatrix}$.

## F. Attaque par force brute

* Combien y a-t-il de matrices $2\times 2$, à coefficients dans $\{0,\ldots,25\}$ ?

* A l'aide d'un programme, déterminer le nombre de matrices de chiffrement admissibles (c'est-à-dire le nombre de matrices $2\times 2$, à coefficients dans $\{0,\ldots,25\}$, inversibles modulo $26$).

Ce nombre n'est pas trop élevé et permet d'envisager une attaque par force brute pour décrypter un texte chiffré avec une matrice qu'on ne connaît pas.

*On parcourt toutes les matrices de chiffrement possibles. Pour chaque matrice, on déchiffre le texte, puis on teste s'il est intelligible (en français). Pour cela, on compte le nombre de mots de la langue française présents dans le texte.*

Sur la page du cours, on trouvera un fichier `dicofr.txt` contenant une liste de référence de mots de la langue française.

- Lire et stocker dans une liste le contenu du fichier `dicofr.txt`.

- Décrypter le message `ZZUCANHLZZVVEMTIAICRESIIUVQEXIIRLDXJAIFXSTCCFKKWRFKNAVIIWHWUHTNVXJGLSTES` sachant qu'il s'agit d'un message écrit en français et chiffré avec un chiffrement de Hill. (Remarque : l'exécution du programme d'attaque peut prendre un certain temps. Pour réduire ce temps, on peut n'utiliser qu'une sous-partie du dictionnaire.)